In [ ]:

import sys, os, multiprocessing, urllib2, csv
from PIL import Image
from StringIO import StringIO

In [ ]:
index_file = "/home/kylecshan/data/index.csv"
index_dir = "/home/kylecshan/data/index/all/"
test_file = "/home/kylecshan/data/test.csv"
test_dir = "/home/kylecshan/data/test/all/"

In [ ]:
#!/usr/bin/python

# Note to Kagglers: This script will not run directly in Kaggle kernels. You
# need to download it and run it on your local machine.

# Downloads images from the Google Landmarks dataset using multiple threads.
# Images that already exist will not be downloaded again, so the script can
# resume a partially completed download. All images will be saved in the JPG
# format with 90% compression quality.

def ParseData(data_file):
    csvfile = open(data_file, 'r')
    csvreader = csv.reader(csvfile)
    key_url_list = [line[:2] for line in csvreader]
    return key_url_list[1:]  # Chop off header




In [ ]:
def DownloadImage(key_url, out_dir):
    (key, url) = key_url
    filename = os.path.join(out_dir, '%s.jpg' % key)

    if os.path.exists(filename):
        #print('Image %s already exists. Skipping download.' % filename)
        return
    
    try:
        response = urllib2.urlopen(url)
        image_data = response.read()
    except:
        #print('Warning: Could not download image %s from %s' % (key, url))
        return
    
    try:
        pil_image = Image.open(StringIO(image_data))
    except:
        print('Warning: Failed to parse image %s' % key)
        return
    

    try:
        pil_image_rgb = pil_image.convert('RGB')
        w,h = pil_image_rgb.size
        mindim = min(w,h)
        size = (int(w*224/mindim), int(h*224/mindim))
        pil_image_rgb = pil_image_rgb.resize(size, resample = Image.BILINEAR)
    except:
        print('Warning: Failed to convert image %s to RGB' % key)
        return
    

    try:
        pil_image_rgb.save(filename, format='JPEG', quality=90)
    except:
        print('Warning: Failed to save image %s' % filename)
        return
    
    return 0


In [ ]:
def Run():
#     if len(sys.argv) != 3:
#         print('Syntax: %s <data_file.csv> <output_dir/>' % sys.argv[0])
#         sys.exit(0)
#     (data_file, out_dir) = sys.argv[1:]

#     if not os.path.exists(out_dir):
#         os.mkdir(out_dir)

    i = 0
    key_url_test = ParseData(test_file)
    for key_url in key_url_test:
        DownloadImage(key_url, test_dir)
        i += 1
        if (i % 5000) == 0:
            print("Finished %d pictures" % i)
        
    key_url_index = ParseData(index_file)
    for key_url in key_url_index:
        DownloadImage(key_url, index_dir)
        i += 1
        if (i % 5000) == 0:
            print("Finished %d pictures" % i)
        
        
#     pool = multiprocessing.Pool(processes=50)
#     pool.map(DownloadImage, key_url_list)


In [ ]:
Run()

In [ ]:
def DownloadImageIndex(key_url):
    index_dir = "/home/kylecshan/data/index/all/"
    DownloadImage(key_url, index_dir)

def Run2():
#     if len(sys.argv) != 3:
#         print('Syntax: %s <data_file.csv> <output_dir/>' % sys.argv[0])
#         sys.exit(0)
#     (data_file, out_dir) = sys.argv[1:]

#     if not os.path.exists(out_dir):
#         os.mkdir(out_dir)

    key_url_index = ParseData(index_file)
    key_url_index = key_url_index[954074:]
    pool = multiprocessing.Pool(processes=50)
    pool.map(DownloadImageIndex, key_url_index)

In [ ]:
Run2()

In [ ]:
key_url_index = ParseData(index_file)
pool = multiprocessing.Pool(processes=50)
pool.map(DownloadImageIndex, key_url_index)